# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 8:51AM,246687,10,0085985991,ISDIN Corporation,Released
1,Sep 19 2022 8:51AM,246687,10,0085986002,ISDIN Corporation,Released
2,Sep 19 2022 8:51AM,246687,10,0085986118,ISDIN Corporation,Released
3,Sep 19 2022 8:51AM,246687,10,0085991124,ISDIN Corporation,Released
4,Sep 19 2022 8:49AM,246684,10,0085986084,ISDIN Corporation,Released
5,Sep 19 2022 8:49AM,246684,10,0085986085,ISDIN Corporation,Released
6,Sep 19 2022 8:49AM,246684,10,0085986086,ISDIN Corporation,Released
7,Sep 19 2022 8:49AM,246684,10,0085986087,ISDIN Corporation,Released
8,Sep 19 2022 8:49AM,246684,10,0085986094,ISDIN Corporation,Released
9,Sep 19 2022 8:49AM,246684,10,0085986093,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,246684,Released,48
9,246685,Executing,1
10,246685,Released,1
11,246686,Released,1
12,246687,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246683,NaN,59.0
246684,NaN,48.0
246685,1.0,1.0
246686,NaN,1.0
246687,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246672,2.0,5.0
246678,0.0,10.0
246679,0.0,60.0
246680,0.0,1.0
246681,0.0,56.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246672,2,5
246678,0,10
246679,0,60
246680,0,1
246681,0,56


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246672,2,5
1,246678,0,10
2,246679,0,60
3,246680,0,1
4,246681,0,56


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246672,2,5
1,246678,,10
2,246679,,60
3,246680,,1
4,246681,,56


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation
4,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation
52,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc."
54,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated"
55,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation
114,Sep 19 2022 8:35AM,246682,10,Emerginnova
120,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation
176,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation
236,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America
237,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation,,4
1,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,,48
2,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc.",1,1
3,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated",,1
4,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,,59
5,Sep 19 2022 8:35AM,246682,10,Emerginnova,,6
6,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation,,56
7,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation,,60
8,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,,1
9,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation,4,
1,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,
2,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc.",1,1
3,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated",1,
4,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,59,
5,Sep 19 2022 8:35AM,246682,10,Emerginnova,6,
6,Sep 19 2022 8:33AM,246681,10,ISDIN Corporation,56,
7,Sep 19 2022 8:29AM,246679,10,ISDIN Corporation,60,
8,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,1,
9,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation,4,
1,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,
2,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc.",1,1
3,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated",1,
4,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,59,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation,4,NaN
1,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,NaN
2,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc.",1,1.0
3,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated",1,NaN
4,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,59,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 8:51AM,246687,10,ISDIN Corporation,4,0.0
1,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,0.0
2,Sep 19 2022 8:47AM,246685,20,"Exact-Rx, Inc.",1,1.0
3,Sep 19 2022 8:47AM,246686,10,"Senseonics, Incorporated",1,0.0
4,Sep 19 2022 8:44AM,246683,10,ISDIN Corporation,59,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1973460,244,0.0
16,246680,1,0.0
19,246672,5,2.0
20,246685,1,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1973460,244,0.0
1,16,246680,1,0.0
2,19,246672,5,2.0
3,20,246685,1,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,244,0.0
1,16,1,0.0
2,19,5,2.0
3,20,1,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,244.0
1,16,Released,1.0
2,19,Released,5.0
3,20,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Executing,0.0,0.0,2.0,1.0
Released,244.0,1.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Executing,0.0,0.0,2.0,1.0
1,Released,244.0,1.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Executing,0.0,0.0,2.0,1.0
1,Released,244.0,1.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()